# Llama 3.2 Vision: 2-Turn Balance-Description Bank Statement Extraction

**Protocol**: Two independent single-turn prompts + Python parsing/filtering

**Key Insight**: Balance-description prompt works for BOTH date-per-row AND date-grouped formats!

---

## Complete Workflow

```
Turn 0: Image + Prompt → Headers (fresh context)
        ↓ (Python pattern matching)
        ↓ (Check if Balance column exists)
Turn 1: Image + Prompt → Balance-Description extraction (fresh context)
        ↓ (Python parsing + filtering)
Schema Fields: TRANSACTION_DATES, LINE_ITEM_DESCRIPTIONS, TRANSACTION_AMOUNTS_PAID
```

### Pipeline Stages:
1. **Turn 0 (LLM)**: Identify column headers from image
2. **Pattern Matching (Python)**: Map headers to concepts (Date, Description, Debit, Credit, Balance)
3. **Turn 1 (LLM)**: Extract using balance-description prompt (works for all date formats)
4. **Python Parsing**: Parse hierarchical output → Filter for debits → Extract schema fields

### Why Balance-Description Works:
- **Anchors extraction to Balance column** - unambiguous reference point
- **Works for date-per-row**: Each transaction gets its date
- **Works for date-grouped**: Date headers naturally map to transactions
- **No format classification needed** - eliminates Turn 0.5 entirely!

### Key Advantages over 3-Turn Table Extraction:
- ✅ **Simpler pipeline** - 2 turns instead of 3
- ✅ **No format classification** - works universally
- ✅ **Balance-anchored** - reduces debit/credit confusion
- ✅ **Hierarchical output** - easier to parse date groupings

In [ ]:
# Cell 1: Imports

# Standard library imports
from pathlib import Path
import random
import re

# Third-party imports
import numpy as np
import torch
from PIL import Image
from transformers import AutoProcessor, MllamaForConditionalGeneration
from IPython.display import display, Markdown, HTML
from common.reproducibility import set_seed
set_seed(42)

# Load the model

In [ ]:
# Cell 3: Load Llama-3.2-Vision model
# Update this path to your local Llama model
model_id = "/home/jovyan/nfs_share/models/Llama-3.2-11B-Vision-Instruct"

print("🔧 Loading Llama-3.2-Vision model...")

from common.llama_model_loader_robust import load_llama_model_robust

model, processor = load_llama_model_robust(
    model_path=model_id,
    use_quantization=False,
    device_map='auto',
    max_new_tokens=4096,  # Increased for balance-description output
    torch_dtype='bfloat16',
    low_cpu_mem_usage=True,
    verbose=True
)

# Add tie_weights() call
try:
    model.tie_weights()
    print("✅ Model weights tied successfully")
except Exception as e:
    print(f"⚠️ tie_weights() warning: {e}")

# Load the image

In [ ]:
# Cell 5: Load bank statement image
# Update this path to your test image
imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/cba_date_grouped.png"

print("📁 Loading image...")
image = Image.open(imageName)

# CRITICAL: Store as list for compatibility
images = [image]

print(f"✅ Image loaded: {image.size}")
print(f"✅ Images list created with {len(images)} image(s)")

# Display the loaded image for visual verification
print("🖼️  Bank statement image:")
display(image)

# Bank Statement Extraction Protocol (2-Turn Balance-Description)
- Turn 0: Identify actual table headers
- Turn 1: Extract using balance-description prompt
- Python: Parse, filter, and extract schema fields

In [ ]:
# Cell 7: Turn 0 - Identify table headers (prompt)
# TURN 0: Identify Table Headers

prompt = """
Look at the transaction table in this bank statement image.

What are the exact column header names used in the transaction table?

List each column header exactly as it appears, in order from left to right.
Do not interpret or rename them - use the EXACT text from the image.
"""

# Create message structure for Llama
messageDataStructure = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {
                "type": "text",
                "text": prompt,
            },
        ],
    }
]

print("💬 TURN 0: Identifying actual table headers")
print("🤖 Generating response with Llama-3.2-Vision...")

In [ ]:
# Cell 8: Turn 0 - Execute and parse headers

textInput = processor.apply_chat_template(
    messageDataStructure, add_generation_prompt=True
)

inputs = processor(images=images, text=textInput, return_tensors="pt").to(model.device)

output = model.generate(
    **inputs,
    max_new_tokens=500,
    do_sample=False,
    temperature=None,
    top_p=None,
)

generate_ids = output[:, inputs['input_ids'].shape[1]:-1]
turn0_response = processor.decode(generate_ids[0], clean_up_tokenization_spaces=False)

# Free memory
del inputs, output, generate_ids
torch.cuda.empty_cache()

print("✅ Response generated successfully!")
print("\n" + "=" * 60)
print("TURN 0 - IDENTIFIED TABLE HEADERS:")
print("=" * 60)
print(turn0_response)
print("=" * 60)

# Parse the identified headers
header_lines = [line.strip() for line in turn0_response.split('\n') if line.strip()]
identified_headers = []

for line in header_lines:
    cleaned = line.lstrip('0123456789.-•* ').strip()
    cleaned = cleaned.replace('**', '').replace('__', '')
    if cleaned.endswith(':'):
        continue
    if len(cleaned) > 40:
        continue
    if cleaned and len(cleaned) > 2:
        identified_headers.append(cleaned)

print(f"\n📋 Parsed {len(identified_headers)} column headers:")
for i, header in enumerate(identified_headers, 1):
    print(f"  {i}. '{header}'")

# Store for subsequent turns
table_headers = identified_headers
print(f"\n✅ Stored table_headers: {table_headers}")

## Pattern Matching: Map Generic Concepts to Actual Headers

Different bank statements use different column names. Use pattern matching to identify:
- Which header represents **Date**
- Which header represents **Description/Details**  
- Which header represents **Debit/Withdrawal**
- Which header represents **Credit/Deposit**
- Which header represents **Balance** (CRITICAL for this approach!)

In [ ]:
# Cell 9: Pattern Matching - Map headers to generic columns

DATE_PATTERNS = ['date', 'day', 'transaction date', 'trans date']
DESCRIPTION_PATTERNS = [
    'description', 'details', 'transaction details', 'trans details',
    'particulars', 'narrative', 'transaction', 'trans'
]
DEBIT_PATTERNS = ['debit', 'debits', 'withdrawal', 'withdrawals', 'paid', 'paid out', 'spent', 'dr']
CREDIT_PATTERNS = ['credit', 'credits', 'deposit', 'deposits', 'received', 'cr']
BALANCE_PATTERNS = ['balance', 'bal', 'running balance']
AMOUNT_PATTERNS = ['amount', 'amt', 'value', 'total']

def match_header(headers, patterns, fallback=None):
    """Match a header using pattern keywords."""
    headers_lower = [h.lower() for h in headers]
    
    # Try exact match first
    for pattern in patterns:
        for i, header_lower in enumerate(headers_lower):
            if pattern == header_lower:
                return headers[i]
    
    # Try substring match
    for pattern in patterns:
        if len(pattern) > 2:
            for i, header_lower in enumerate(headers_lower):
                if pattern in header_lower:
                    return headers[i]
    
    return fallback

# Perform pattern matching
date_col = match_header(table_headers, DATE_PATTERNS, fallback=table_headers[0] if table_headers else 'Date')
desc_col = match_header(table_headers, DESCRIPTION_PATTERNS, fallback=table_headers[1] if len(table_headers) > 1 else 'Description')
amount_col = match_header(table_headers, AMOUNT_PATTERNS, fallback=None)
debit_col = match_header(table_headers, DEBIT_PATTERNS, fallback=amount_col if amount_col else 'Debit')
credit_col = match_header(table_headers, CREDIT_PATTERNS, fallback=amount_col if amount_col else 'Credit')
balance_col = match_header(table_headers, BALANCE_PATTERNS, fallback=None)

print("=" * 60)
print("PATTERN MATCHING RESULTS:")
print("=" * 60)
print(f"📋 Extracted Headers: {table_headers}")
print(f"\n🔍 Mapped Columns:")
print(f"  Date        → '{date_col}'")
print(f"  Description → '{desc_col}'")
print(f"  Debit       → '{debit_col}'")
print(f"  Credit      → '{credit_col}'")
print(f"  Balance     → '{balance_col}'")

# CRITICAL: Check if Balance column exists
has_balance = balance_col is not None and balance_col in table_headers
print(f"\n🎯 Balance column detected: {'✅ YES' if has_balance else '❌ NO'}")

if not has_balance:
    print("⚠️  WARNING: No balance column found. This notebook requires a balance column.")
    print("   Consider using the 3-turn table extraction notebook instead.")

## Turn 1: Balance-Description Extraction

This is the key innovation - instead of extracting a markdown table, we ask the model to list all balances with their associated transaction details.

**Prompt Pattern:**
```
List all the balances in the {balance_col} column, including:
- Date from the Date Header of the balance
- {desc_col}
- {debit_col} Amount or "NOT_FOUND"
- {credit_col} Amount or "NOT_FOUND"
```

This produces hierarchical output that's easy to parse and works for both date formats!

In [ ]:
# Cell 10: Turn 1 - Balance-Description Extraction Prompt

if has_balance:
    extraction_prompt = f"""List all the balances in the {balance_col} column, including:
- Date from the Date Header of the balance
- {desc_col}
- {debit_col} Amount or "NOT_FOUND"
- {credit_col} Amount or "NOT_FOUND" """
    
    print("📝 TURN 1: Balance-Description Extraction")
    print("=" * 60)
    print("Extraction Prompt:")
    print(extraction_prompt)
    print("=" * 60)
else:
    print("❌ Cannot proceed - no balance column detected.")
    extraction_prompt = None

In [ ]:
# Cell 11: Turn 1 - Execute extraction (INDEPENDENT, fresh context)

if extraction_prompt:
    # CRITICAL: Create FRESH message structure (NOT appending to conversation history)
    messageDataStructure_turn1 = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": extraction_prompt}
            ]
        }
    ]
    
    print("🤖 Generating response with Llama-3.2-Vision...")
    
    textInput = processor.apply_chat_template(
        messageDataStructure_turn1, add_generation_prompt=True
    )
    
    inputs = processor(images=images, text=textInput, return_tensors="pt").to(model.device)
    
    output = model.generate(
        **inputs,
        max_new_tokens=4096,  # Increased for balance-description output
        do_sample=False,
        temperature=None,
        top_p=None,
    )
    
    generate_ids = output[:, inputs['input_ids'].shape[1]:-1]
    extraction_response = processor.decode(generate_ids[0], clean_up_tokenization_spaces=False)
    
    # Free memory
    del inputs, output, generate_ids
    torch.cuda.empty_cache()
    
    print("\n✅ Turn 1 extraction complete!")
    print(f"\n📊 Response length: {len(extraction_response)} characters")
    print("\n" + "=" * 60)
    print("TURN 1 - BALANCE-DESCRIPTION EXTRACTION:")
    print("=" * 60)
    print(extraction_response)
    print("=" * 60)
else:
    print("❌ Skipping extraction - no valid prompt.")
    extraction_response = None

## Python Parsing: Balance-Description Response

Parse the hierarchical balance-description output into transaction rows.

Expected format:
```
1. **Thu 04 Sep 2025**
   - Description: Direct Debit DOMINO'S PTY LTD
   - Debit: $117.57
   - Balance: $8586.28 CR
```

In [ ]:
# Cell 12: Parse balance-description response

def parse_balance_description_response(response_text, date_col, desc_col, debit_col, credit_col, balance_col):
    """
    Parse the hierarchical balance-description response into transaction rows.
    
    Returns list of dicts with standardized column names.
    """
    rows = []
    current_date = None
    current_transaction = {}
    
    lines = response_text.strip().split("\n")
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        
        # Check for date header (numbered item with bold date)
        # Pattern: "1. **Thu 04 Sep 2025**" or "1. Thu 04 Sep 2025"
        date_match = re.match(r"^\d+\.\s*\*?\*?([A-Za-z]{3}\s+\d{1,2}\s+[A-Za-z]{3}\s+\d{4})\*?\*?", line)
        if not date_match:
            # Also try without day name: "1. **04 Sep 2025**"
            date_match = re.match(r"^\d+\.\s*\*?\*?(\d{1,2}\s+[A-Za-z]{3}\s+\d{4})\*?\*?", line)
        if not date_match:
            # Also try DD/MM/YYYY format: "1. **03/05/2025**"
            date_match = re.match(r"^\d+\.\s*\*?\*?(\d{1,2}/\d{1,2}/\d{4})\*?\*?", line)
        
        if date_match:
            # Save previous transaction if exists
            if current_transaction and current_date:
                current_transaction[date_col] = current_date
                rows.append(current_transaction)
                current_transaction = {}
            
            current_date = date_match.group(1).strip()
            continue
        
        # Check for field lines
        # Pattern: "- Description: ..." or "   - Debit: ..."
        field_match = re.match(r"^\s*-\s*(\w+):\s*(.+)$", line)
        if field_match:
            field_name = field_match.group(1).strip().lower()
            field_value = field_match.group(2).strip()
            
            # Map field names to column names
            if field_name == "description":
                # If we already have a description, this is a new transaction under same date
                if desc_col in current_transaction and current_transaction[desc_col]:
                    # Save current transaction
                    if current_date:
                        current_transaction[date_col] = current_date
                    rows.append(current_transaction)
                    current_transaction = {}
                current_transaction[desc_col] = field_value
            
            elif field_name == "debit" or field_name == debit_col.lower() or field_name == "withdrawal":
                current_transaction[debit_col] = field_value
            
            elif field_name == "credit" or field_name == credit_col.lower() or field_name == "deposit":
                current_transaction[credit_col] = field_value
            
            elif field_name == "balance":
                current_transaction[balance_col] = field_value
            
            elif field_name == "amount":
                # Generic amount - put in debit by default
                if debit_col not in current_transaction:
                    current_transaction[debit_col] = field_value
    
    # Don't forget the last transaction
    if current_transaction and current_date:
        current_transaction[date_col] = current_date
        rows.append(current_transaction)
    
    return rows


def parse_markdown_table(markdown_text):
    """Fallback: Parse markdown table into list of dictionaries."""
    lines = [line.strip() for line in markdown_text.strip().split('\n') if line.strip()]
    
    header_idx = None
    for i, line in enumerate(lines):
        if '|' in line:
            cleaned = line.replace('|', '').replace('-', '').replace(' ', '')
            if cleaned:
                header_idx = i
                break
    
    if header_idx is None:
        return []
    
    header_line = lines[header_idx]
    header_parts = [h.strip() for h in header_line.split('|')]
    if header_parts and header_parts[0] == '':
        header_parts = header_parts[1:]
    if header_parts and header_parts[-1] == '':
        header_parts = header_parts[:-1]
    headers = [h for h in header_parts if h]
    
    rows = []
    for line in lines[header_idx + 1:]:
        if '|' not in line:
            continue
        cleaned = line.replace('|', '').replace('-', '').replace(' ', '').replace(':', '')
        if not cleaned:
            continue
        value_parts = [v.strip() for v in line.split('|')]
        if value_parts and value_parts[0] == '':
            value_parts = value_parts[1:]
        if value_parts and value_parts[-1] == '':
            value_parts = value_parts[:-1]
        if len(value_parts) == len(headers):
            rows.append(dict(zip(headers, value_parts)))
    
    return rows


# Parse the extraction response
if extraction_response:
    # Try balance-description format first
    all_rows = parse_balance_description_response(
        extraction_response, date_col, desc_col, debit_col, credit_col, balance_col
    )
    
    # Fallback to markdown table if balance-description parsing failed
    if not all_rows and "|" in extraction_response:
        print("⚠️  Fallback: parsing as markdown table")
        all_rows = parse_markdown_table(extraction_response)
    
    print(f"\n📊 Parsed {len(all_rows)} total rows")
    
    # Display parsed rows
    print("\n" + "=" * 60)
    print("PARSED TRANSACTIONS:")
    print("=" * 60)
    for i, row in enumerate(all_rows[:10]):  # Show first 10
        print(f"\n{i+1}. {row}")
    if len(all_rows) > 10:
        print(f"\n... and {len(all_rows) - 10} more rows")
else:
    all_rows = []

## Filter for Debit Transactions

For tax purposes, we only want transactions where the taxpayer PAID money (debits/withdrawals).

In [ ]:
# Cell 13: Filter for debit transactions

def parse_amount(value):
    """Extract numeric value from formatted currency string."""
    if not value or value.strip() == "":
        return 0.0
    cleaned = value.replace("$", "").replace(",", "").replace("CR", "").replace("DR", "").strip()
    try:
        return float(cleaned)
    except ValueError:
        return 0.0


def is_non_transaction_row(row, desc_col):
    """Check if this row is NOT an actual transaction."""
    desc = row.get(desc_col, "").strip().upper()
    if "OPENING BALANCE" in desc:
        return True
    if "CLOSING BALANCE" in desc:
        return True
    if "BROUGHT FORWARD" in desc:
        return True
    if "CARRIED FORWARD" in desc:
        return True
    return False


def filter_debit_transactions(rows, debit_col, desc_col=None):
    """Filter rows to only those with actual debit transactions (amount > 0)."""
    debit_rows = []
    for row in rows:
        debit_value = row.get(debit_col, "").strip()
        
        if not debit_value:
            continue
        
        # Skip NOT_FOUND values
        if debit_value.upper() == "NOT_FOUND":
            continue
        
        # Parse amount and check if > 0
        amount = parse_amount(debit_value)
        if amount <= 0:
            continue
        
        if desc_col and is_non_transaction_row(row, desc_col):
            continue
        
        debit_rows.append(row)
    
    return debit_rows


# Filter to debit transactions
debit_rows = filter_debit_transactions(all_rows, debit_col, desc_col)

print(f"\n📊 Filtered to {len(debit_rows)} debit transactions")
print("\n" + "=" * 60)
print("DEBIT TRANSACTIONS ONLY:")
print("=" * 60)
for i, row in enumerate(debit_rows):
    date = row.get(date_col, "N/A")
    desc = row.get(desc_col, "N/A")
    amount = row.get(debit_col, "N/A")
    print(f"{i+1}. [{date}] {desc} - {amount}")

## Extract Schema Fields

Convert parsed debit transactions to universal.yaml schema format:
- `TRANSACTION_DATES`: Pipe-delimited dates
- `LINE_ITEM_DESCRIPTIONS`: Pipe-delimited descriptions
- `TRANSACTION_AMOUNTS_PAID`: Pipe-delimited amounts

In [ ]:
# Cell 14: Extract schema fields

def extract_schema_fields(debit_rows, date_col, desc_col, debit_col, all_rows=None):
    """Extract fields in universal.yaml schema format."""
    if not debit_rows:
        return {
            "DOCUMENT_TYPE": "BANK_STATEMENT",
            "STATEMENT_DATE_RANGE": "NOT_FOUND",
            "TRANSACTION_DATES": "NOT_FOUND",
            "LINE_ITEM_DESCRIPTIONS": "NOT_FOUND",
            "TRANSACTION_AMOUNTS_PAID": "NOT_FOUND",
        }
    
    # Extract debit transaction fields
    debit_dates = []
    descriptions = []
    amounts = []
    
    for row in debit_rows:
        date = row.get(date_col, "").strip()
        desc = row.get(desc_col, "").strip()
        amount = row.get(debit_col, "").strip()
        
        if date:
            debit_dates.append(date)
        if desc:
            descriptions.append(desc)
        if amount:
            amounts.append(amount)
    
    # Calculate date range from ALL transactions (not just debits)
    date_range = "NOT_FOUND"
    rows_for_range = all_rows if all_rows is not None else debit_rows
    all_dates = [row.get(date_col, "").strip() for row in rows_for_range]
    all_dates = [d for d in all_dates if d]
    if all_dates:
        date_range = f"{all_dates[0]} - {all_dates[-1]}"
    
    return {
        "DOCUMENT_TYPE": "BANK_STATEMENT",
        "STATEMENT_DATE_RANGE": date_range,
        "TRANSACTION_DATES": " | ".join(debit_dates) if debit_dates else "NOT_FOUND",
        "LINE_ITEM_DESCRIPTIONS": " | ".join(descriptions) if descriptions else "NOT_FOUND",
        "TRANSACTION_AMOUNTS_PAID": " | ".join(amounts) if amounts else "NOT_FOUND",
    }


# Extract schema fields
schema_fields = extract_schema_fields(debit_rows, date_col, desc_col, debit_col, all_rows=all_rows)

print("\n" + "=" * 60)
print("EXTRACTED SCHEMA FIELDS:")
print("=" * 60)
for field, value in schema_fields.items():
    # Truncate long values for display
    display_value = str(value)[:100] + "..." if len(str(value)) > 100 else str(value)
    print(f"\n{field}:")
    print(f"  {display_value}")

## Summary

This 2-turn balance-description approach:
1. ✅ **Eliminated Turn 0.5** - No date format classification needed
2. ✅ **Works universally** - Same prompt for date-per-row AND date-grouped
3. ✅ **Balance-anchored** - Uses balance column as reference point
4. ✅ **Simpler pipeline** - Just 2 LLM calls instead of 3

In [ ]:
# Cell 15: Summary statistics

print("\n" + "=" * 60)
print("📊 EXTRACTION SUMMARY")
print("=" * 60)
print(f"\n🔧 Method: 2-Turn Balance-Description")
print(f"📋 Headers detected: {len(table_headers)}")
print(f"💰 Balance column: {balance_col}")
print(f"📝 Total transactions parsed: {len(all_rows)}")
print(f"💸 Debit transactions: {len(debit_rows)}")
print(f"\n✅ Pipeline complete!")